In [1]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 4.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 264 kB 18.5 MB/s 
     |████████████████████████████████| 124 kB 19.4 MB/s 
     |████████████████████████████████| 1.8 MB 20.5 MB/s 
     |████████████████████████████████| 8.0 MB 30.8 MB/s 
     |████████████████████████████████| 1.2 MB 66.4 MB/s 
     |████████████████████████████████| 2.6 MB 44.9 MB/s 
     |████████████████████████████████| 76 kB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 54.1 MB/s 
     |████████████████████████████████| 895 kB 62.6 MB/s 
     |████████████████████████████████| 636 kB 59.6 MB/s 
     |████████████████████████████████| 170 kB 50.8 MB/s 
     |████████████████████████████████| 138 kB 73.0 MB/s 
     |████████████████████████████████| 97 kB 7.4 MB/s 
     |████████████████████████████████| 133 kB 50.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |█████████████████

In [1]:
import pandas as pd
df=pd.read_csv("/content/bbc-text.csv",nrows=2000)
print(pd.Categorical(df["category"].unique()))
print(pd.Categorical(df["category"].unique()).codes)
df["category"] = pd.Categorical(df["category"]).codes
df

['tech', 'business', 'sport', 'entertainment', 'politics']
Categories (5, object): ['business', 'entertainment', 'politics', 'sport', 'tech']
[4 0 3 1 2]


,category,text
0,4,tv future in the hands of viewers with home th...
1,0,worldcom boss left books alone former worldc...
2,3,tigers wary of farrell gamble leicester say ...
3,3,yeading face newcastle in fa cup premiership s...
4,1,ocean s twelve raids box office ocean s twelve...
...,...,...
1995,3,cole refuses to blame van persie ashley cole h...
1996,3,brentford v southampton griffin park tuesday ...
1997,3,clijsters set for february return tennis star ...
1998,0,cannabis hopes for drug firm a prescription ca...


In [ ]:
# df.shape[0]

2000

In [ ]:
# len(pd.Categorical(df["category"].unique()))

5

In [ ]:
# i=0
# df.query('category == @i').shape[0]

455

In [2]:
weights=[]
total_class = len(pd.Categorical(df["category"].unique()))
for i in range(0,total_class):
  label_spec_len = df.query('category == @i').shape[0]
  label_weight = df.shape[0] / (total_class * label_spec_len)
  weights.append(label_weight)
weights

[0.8791208791208791,
 1.13314447592068,
 1.0638297872340425,
 0.8547008547008547,
 1.1494252873563218]

In [3]:
df.rename(columns={"category": "label",},inplace=True)
df

,label,text
0,4,tv future in the hands of viewers with home th...
1,0,worldcom boss left books alone former worldc...
2,3,tigers wary of farrell gamble leicester say ...
3,3,yeading face newcastle in fa cup premiership s...
4,1,ocean s twelve raids box office ocean s twelve...
...,...,...
1995,3,cole refuses to blame van persie ashley cole h...
1996,3,brentford v southampton griffin park tuesday ...
1997,3,clijsters set for february return tennis star ...
1998,0,cannabis hopes for drug firm a prescription ca...


In [4]:
train_df = df[['text', 'label']]
eval_df = df[['text', 'label']]

In [5]:
from simpletransformers.classification import ClassificationModel
import torch
args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",
    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "logging_steps": 50,
    "save_steps": 2000,
    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": False,
    "save_model_every_epoch":False,
    # "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
}
model = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    num_labels=total_class,
    # weight=torch.Tensor(weights).to("cuda"),
    weight=weights,
    args=args,
    use_cuda=True,
)
model.train_model(train_df)

2021-08-06 07:39:57.018976: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/250 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


(250, 0.33091044902801514)

In [6]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]